# Libraries

In [3]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from scipy.stats import gamma

output_notebook()

Loading BokehJS ...

# Helper functions

In [4]:
def dec2bin (N_bits,number):
    return np.fromiter((a for a in np.binary_repr(int(number),N_bits)), dtype=int)[::-1]

def bin2dec (N_bits,number):
    return np.dot(number,np.fromiter((2**x for x in range(N_bits)), dtype=int))

def LFSR_20bit (lfsr_i):
        lfsr_o = np.roll(lfsr_i,1)
        lfsr_o[3] = lfsr_o[3]^lfsr_o[0]
        return lfsr_o
    
def V_in(t, input_type):
    if input_type[0]=='pulses':
        if input_type[1] == 'close':
            t2, t1 = 26, 24
        else:
            t2, t1 = 46, 44
        if (t<19 and t>=17) or (t<t2 and t>=t1):
            V = 120                                      # constant input
        else:
            V = 0
    elif input_type[0] == 'constant':
        V = 20
    return V

# Models

In [47]:
def LIF(V_reset, V_thres, tau, dur, R, Ek, g_max, tau_r, input_type=('pulses', 'close')):
    # LIF neuron
    # tau is the time constant in units of dt (integer)

    # initialisation
    g = 0
    a = tau/(tau + R*g + 1.0)
    b = tau_r/(tau_r + 1.0)
    V_mem = V_reset                             # reset initial voltage
    Vs = V_mem                                  # store V_int in trace for plotting
    gs = g

    for t in range(dur):                        # simulate dur time steps
        g = b*g
        V_mem = a*V_mem + (1-a)*V_in(t, input_type) + R*g*Ek/(1+R*g+tau)         # LPF equation
        Vs = np.append(Vs,V_mem)                   # append V_mem to trace
        gs = np.append(gs, g)
        if V_mem >= V_thres:                    # check if V_mem > threshold
            g = g_max
            V_mem = V_reset                     # reset V_mem
            Vs[-1] = 40                         # plot spike

    return Vs, gs

def LIFn(V_reset, V_thres, tau, dur, R, Ek, g_max, tau_r, input_type=('pulses', 'close')):
    # LIF neuron
    # tau is the time constant in units of dt (integer)

    # initialisation
    g = 0
    a = tau/(tau + R*g + 1.0)
    b = tau_r/(tau_r + 1.0)
    V_mem = V_reset                             # reset initial voltage
    Vs = V_mem                                  # store V_int in trace for plotting
    gs = g
    ST = []

    for t in range(dur):                        # simulate dur time steps
        g = b*g
        V_mem = a*V_mem + (1-a)*V_in(t, input_type) + R*g*Ek/(1+R*g+tau)        # LPF equation
        V_mem += np.random.randn()                        # additive Gaussian (Normal) noise
        Vs = np.append(Vs,V_mem)                   # append V_mem to trace
        gs = np.append(gs, g)
        if V_mem >= V_thres:                    # check if V_mem > threshold
            g = g_max
            V_mem = V_reset                     # reset V_mem
            Vs[-1] = 40                         # plot spike
            ST = np.append(ST,t)                   # record spike time

    return Vs, ST, gs

def SLIF(V_reset, V_thres, tau, seed, V_bits, dur, R, Ek, g_max, tau_r, input_type=('pulses', 'close')):
    # stochastic LIF neuron
    # tau is the time constant in units of dt (integer)

    # initialisation
    g = 0
    a = tau/(tau + R*g + 1.0)
    b = tau_r/(tau_r + 1.0)
    lfsr = LFSR_20bit(dec2bin(20,seed))         # load seed
    V_temp = V_reset             # convert to binary
    V_mem = V_reset                             # reset initial voltage
    Vs = [V_mem]                                  # store V_int in trace for plotting
    gs = g
    ST = []                                     # list of spike times

    for t in range(dur):                        # simulate dur time steps
        lfsr = LFSR_20bit(lfsr)                 # clock the shift register
        r = bin2dec(20,lfsr)/2.**20             # generate the random fraction
        r2 = bin2dec(20,lfsr)/2.**20
        V_mem = V_temp
        Vin = V_in(t, input_type)                           # read filter input
        g = int(b*g +r2)
        V_mem = int((V_mem-Vin)*a + Vin + r + R*g*Ek/(1+R*g+tau))    # LPF equation
        Vs = np.append(Vs,V_mem)                   # append V_mem to  trace
        gs = np.append(gs, g)
        if V_mem >= V_thres:                    # check if V_mem > threshold
            g = g_max
            V_mem = V_reset                     # reset V_mem
            Vs[-1] = 40                         # plot spike
            ST = np.append(ST,t)                   # record spike time
        V_temp = V_mem          # convert back to binary (4 bits)

    return Vs, ST, gs

def SLIF2(V_reset, V_thres, tau, tau_r, seed, V_bits, dur, input_type=('pulses', 'close')):
    # stochastic LIF neuron
    # tau is the time constant in units of dt (integer)
    # Refractory period is just voltage clipped to Vr for a while

    # initialisation
    v_init = 0
    refrac_tau = tau_r
    a = tau/(tau + 1.0)
    b = refrac_tau/(refrac_tau + 1.0)
    lfsr = LFSR_20bit(dec2bin(20,seed))         # load seed
    # TODO V_bin = dec2bin(V_bits,V_reset)             # convert to binary
    V_mem = v_init                              # reset initial voltage
    V_temp = V_mem
    Vs = V_mem                                  # store V_int in trace for plotting
    ST = []                                     # list of spike times

    for t in range(dur):                        # simulate dur time steps
        lfsr = LFSR_20bit(lfsr)                 # clock the shift register
        r = bin2dec(20,lfsr)/2.**20             # generate the random fraction
        V_mem = V_temp# TODO V_mem = bin2dec(V_bits, V_bin)          # convert binary stored state to integer
        Vin = V_in(t, input_type)
        
        if V_mem < v_init:
            V_mem = int(V_mem*b + r)
            Vs = np.append(Vs,V_mem)
        else:
            V_mem = int((V_mem-Vin)*a + Vin + r)    # LPF equation
            Vs = np.append(Vs,V_mem)                   # append V_mem to  trace
            if V_mem >= V_thres:                    # check if V_mem > threshold
                V_mem = V_reset                     # reset V_mem
                Vs[-1] = V_reset # TODO 40                         # plot spike
                ST = np.append(ST, t)                   # record spike time
        V_temp = V_mem # TODO V_bin = dec2bin(V_bits, V_mem)          # convert back to binary (4 bits)

    return Vs, ST

To emulate a hyperpolarization, a potassium channel was added: $$C\frac{dv_m}{dt}=\frac{v_{in}-v_m}{R_l}+\frac{E_k-v_m}{R_k}$$

$$\tau\frac{dv_m}{dt}=v_m(-1-R_lg_k)+R_lg_kE_k+v_{in}$$

To discretize this equation as shown in the previous notebook, we can do the following
$$v_m=\frac{\tau dv_m/dt-R_lg_kE_k-v_{in}}{(-1-R_lg_k)}$$
$$v_m[t+1]=\frac{\tau}{(-1-R_lg_k)}\frac{(v_m[t+1]-v_m[t])}{\Delta t}+\frac{R_lg_kE_k}{1+R_lg_k}+\frac{v_{in}}{1+R_lg_k}$$

After further manipulations, we can see that
$$v_m[t+1]=\frac{\tau}{\Delta t(1+R_lg_k)+\tau}v_m[t]+\frac{\Delta tR_lg_kE_k}{\Delta t(1+R_lg_k)+\tau}+\frac{\Delta tv_{in}}{\Delta t(1+R_lg_k)+\tau}$$

With $\Delta t=1$, we can see that it is almost as the equation used in the previous notebook, but in this case $a=\frac{\tau}{\tau+(1+R_lg_k)}$ and there is the aditional term $\frac{R_lg_kE_k}{1+R_lg_k+\tau}$

In [159]:
fig = figure(width=700, height=450)
Vs, g = LIF(V_reset = 0, V_thres = 16, tau = 12, R=6, Ek=-5, dur = 60, g_max=10, tau_r=5e-1)
fig.line(range(0, len(Vs)), Vs, color='black', line_width=2)
fig.line(range(0, len(Vs)), g, color='red', line_width=2)
show(fig)

In [160]:
fig = figure(width=700, height=450)
Vs, _, g = LIFn(V_reset = 0, V_thres = 16, tau = 12, R=6, Ek=-5, dur = 60, g_max=10, tau_r=5e-1)
fig.line(range(0, len(Vs)), Vs, color='black', line_width=2)
fig.line(range(0, len(Vs)), g, color='red', line_width=2)
show(fig)

In [161]:
fig = figure(width=700, height=450)
Vs, _, g = SLIF(V_reset = 0, V_thres = 16, tau = 12, R=6, Ek=-8, dur = 60, g_max=10, tau_r=5e-1, seed=123, V_bits=4)
fig.line(range(0, len(Vs)), Vs, color='black', line_width=2)
fig.line(range(0, len(Vs)), g, color='red', line_width=2)
show(fig)

In [163]:
fig = figure(width=700, height=450)
Vs, _, g = SLIF(V_reset = 0, V_thres = 16, tau = 12, R=6, Ek=-5, dur = 200, g_max=10, tau_r=5e-1, seed=123, V_bits=4, input_type=('constant', ''))
fig.line(range(0, len(Vs)), Vs, color='black', line_width=2)
show(fig)

In [165]:
fig = figure(width=700, height=450)
Vs, ST, _ = SLIF(V_reset = 0, V_thres = 16, tau = 12, R=6, Ek=-5, dur = 10000, g_max=10, tau_r=5e-1, seed=123, V_bits=4, input_type=('constant',''))
ISI = ST[1:-1] - ST[0:-2]
hist, edges = np.histogram(ISI, bins=40, range=(0,100))
fig.quad(top=hist, bottom=0, fill_color='black', line_color='grey', left=edges[:-1], right=edges[1:])
fig.line(edges, 1200*gamma.pdf(edges, 2, scale=4, loc=16), line_width=8, alpha=0.7)
show(fig)

Another option is explicity declare a period of time after an action potential in which the membrane potential is clamped to a certain value. This is shown below:

In [53]:
fig = figure(width=700, height=450)
Vs, ST = SLIF2(V_reset = -50, V_thres = 16, seed=12345, V_bits=4, tau = 19, tau_r = 10, dur = 200, input_type=('constant',''))
fig.line(range(0, len(Vs)), Vs, color='black', line_width=2)
show(fig)

In [43]:
fig = figure(width=700, height=450)
(Vs, ST) = SLIF2(V_reset = -5, V_thres = 16, seed=12345, V_bits=4, tau = 19, tau_r = 10, dur = 10000, input_type=('constant',''))
ISI = ST[1:-1] - ST[0:-2]
hist, edges = np.histogram(ISI, bins=30, range=(0,100))
fig.quad(top=hist, bottom=0, fill_color='black', line_color='grey', left=edges[:-1], right=edges[1:])
fig.line(edges, 1000*gamma.pdf(edges, 4, scale=4, loc=22), line_width=8, alpha=0.7)
show(fig)